<b>For faster work</b>, the results of several steps are saved to files:

<i>dna2vec_data/word2vec_49000genes_bpe_size300.model</i>

<i>util_data/bpe_data_with_gene_seq.csv</i>

<i>util_data/genes_1000_pre_vec.csv</i>

So, one can load these files not to wait for long calculations

In [25]:
import os
from timeit import default_timer as timer

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import json
from Bio import SeqIO
from gensim.models import Word2Vec
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, homogeneity_score, completeness_score
from sklearn.metrics import v_measure_score, adjusted_rand_score, adjusted_mutual_info_score

import multiprocessing

### Read .fasta files (49000 genes)

In [3]:
path_gene_nuc = '../../sf_data/prodigal_output/gene_nuc/'
organisms_names = [f[:f.index('_', f.index('_') + 1)] for f in os.listdir(path_gene_nuc)]
full_path_gene_nuc = [path_gene_nuc + f for f in os.listdir(path_gene_nuc)]

In [4]:
organisms = [SeqIO.parse(gene_nuc_file, 'fasta') for gene_nuc_file in full_path_gene_nuc]
org_gene_lst = []

for (id_org, organism), org_name in zip(enumerate(organisms), organisms_names):
    for gene_record in organism:
        org_gene_lst.append(('_'.join((str(id_org), org_name)), gene_record.id,
                             str(gene_record.seq).replace('N', '')))

In [5]:
del path_gene_nuc
del organisms
del organisms_names
del full_path_gene_nuc

In [6]:
len(org_gene_lst)

49807

In [7]:
org_gene_lst[1]

('0_Bacillus_amyloliquefaciens',
 'NC_014551.1_2',
 'ATGAAATTTACGATTCAAAAAGATCGTTTAGTCGAAAGTGTACAAGACGTTCTAAAAGCTGTTTCATCCAGAACAACAATTCCCATTCTTACCGGTATTAAGATCGTAGCCTCAGATGAAGGAGTATCATTTACAGGAAGCGACTCTGATATTTCAATCGAGTCATTCATTCCAAAAGAAGAAGATGATAAGGAAATCGTGACGATCGATCAGCCGGGAAGTATCGTGCTGCAAGCCCGTTTCTTCAGTGAAATCGTTAAAAAACTTCCGATGGCGACTGTTGAAATCGAAGTTCAAAATCAATATTTAACGATTATCCGCTTCGGAAAAGCCGAATTTAATTTAAACGGCCTTGATGCAGATGAATATCCTCATCTTCCGCAAATCGAAGAGCATCATGCCATTCAGATTCCGACGGACCTTTTGAAAAACCTAATCCGTCAAACCGTGTTCGCTGTGTCCGCCTCAGAAACACGCCCTATCTTGACAGGTGTAAACTGGAAAGTGGAAAACGGTGAATTATTATGCACTGCCACGGATAGCCATCGACTCGCTTTAAGAAAGGCGAAGCTTGATATTCCGGAAGACAGCTCTTATAACGTCGTCATCCCCGGAAAAAGCTTAACCGAGCTGAGCAAAATTCTTGATGACAGCCAAGAACTGGTAGATATCGTTATTACAGAAACGCAAGTTCTGTTTAAAGCGAAAAACGTATTGTTCTTCTCAAGGCTGCTTGACGGAAATTATCCGGACACGACCAGCTTAATTCCGCAGGAAAGCAAAACAGAAATCGTCGTGAACACGAAGGAATTTTTACAGGCGATAGACCGTGCTTCTTTATTGGCGAGAGAAGGCCGCAACAACGTTGTAAAGCTTTCTGCCAAACCGGCAGACTCGCTTGAGATTTCATCAAACTCTCCGGAAATCGGGAAAGTAGTCGAAGCTGT

### Read bpe encoded genes

In [8]:
with open('encoded_genes_bpe', 'r') as bpe_file:
    bpe_data = json.load(bpe_file)

In [9]:
type(bpe_data)

list

In [10]:
len(bpe_data)

49806

In [82]:
#bpe_data[0]

In [12]:
all_genes_words_bpe = [item[2] for item in bpe_data]

In [13]:
type(all_genes_words_bpe), type(all_genes_words_bpe[0])

(list, list)

### Explore bpe_data

In [14]:
ngram_lengths = [len(ngram)
                 for gene_ngrams in all_genes_words_bpe
                 for ngram in gene_ngrams]

In [15]:
len(ngram_lengths)

7058023

In [16]:
min(ngram_lengths), ngram_lengths.count(min(ngram_lengths))

(1, 1808)

In [17]:
max(ngram_lengths), ngram_lengths.count(max(ngram_lengths))

(16, 1326)

In [18]:
for i in range(1,17):
    print(f'{i}: {ngram_lengths.count(i)}')

1: 1808
2: 56821
3: 274296
4: 872819
5: 1459433
6: 1556682
7: 1295227
8: 893216
9: 472311
10: 135843
11: 24610
12: 7715
13: 2663
14: 1801
15: 1452
16: 1326


### Add gene_seq to bpe_data

In [19]:
org_gene_lst[1][1], bpe_data[0][1]

('NC_014551.1_2', 'NC_014551.1_2')

In [20]:
org_gene_lst[1][2]

'ATGAAATTTACGATTCAAAAAGATCGTTTAGTCGAAAGTGTACAAGACGTTCTAAAAGCTGTTTCATCCAGAACAACAATTCCCATTCTTACCGGTATTAAGATCGTAGCCTCAGATGAAGGAGTATCATTTACAGGAAGCGACTCTGATATTTCAATCGAGTCATTCATTCCAAAAGAAGAAGATGATAAGGAAATCGTGACGATCGATCAGCCGGGAAGTATCGTGCTGCAAGCCCGTTTCTTCAGTGAAATCGTTAAAAAACTTCCGATGGCGACTGTTGAAATCGAAGTTCAAAATCAATATTTAACGATTATCCGCTTCGGAAAAGCCGAATTTAATTTAAACGGCCTTGATGCAGATGAATATCCTCATCTTCCGCAAATCGAAGAGCATCATGCCATTCAGATTCCGACGGACCTTTTGAAAAACCTAATCCGTCAAACCGTGTTCGCTGTGTCCGCCTCAGAAACACGCCCTATCTTGACAGGTGTAAACTGGAAAGTGGAAAACGGTGAATTATTATGCACTGCCACGGATAGCCATCGACTCGCTTTAAGAAAGGCGAAGCTTGATATTCCGGAAGACAGCTCTTATAACGTCGTCATCCCCGGAAAAAGCTTAACCGAGCTGAGCAAAATTCTTGATGACAGCCAAGAACTGGTAGATATCGTTATTACAGAAACGCAAGTTCTGTTTAAAGCGAAAAACGTATTGTTCTTCTCAAGGCTGCTTGACGGAAATTATCCGGACACGACCAGCTTAATTCCGCAGGAAAGCAAAACAGAAATCGTCGTGAACACGAAGGAATTTTTACAGGCGATAGACCGTGCTTCTTTATTGGCGAGAGAAGGCCGCAACAACGTTGTAAAGCTTTCTGCCAAACCGGCAGACTCGCTTGAGATTTCATCAAACTCTCCGGAAATCGGGAAAGTAGTCGAAGCTGTGCTGGCTGACCAGATTGACGGCGAGGAATTAAATATTTCATTCAGTCCAAAA

In [101]:
# wery long! try to rewrite!
# I will save the result not to wait
#bpe_data_with_gene_seq = [(bpe[0], bpe[1], org_gene[2], bpe[2]) for bpe in bpe_data for org_gene in org_gene_lst if bpe[1] == org_gene[1]]

In [104]:
#pd.DataFrame(bpe_data_with_gene_seq).to_csv('util_data/bpe_data_with_gene_seq.csv', header=False, index=False)

In [21]:
# just read that data
bpe_data_with_gene_seq = pd.read_csv('util_data/bpe_data_with_gene_seq.csv', header=None, index_col=None)

In [22]:
bpe_data_with_gene_seq.head()

,0,1,2,3
0,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,NC_014551.1_2,ATGAAATTTACGATTCAAAAAGATCGTTTAGTCGAAAGTGTACAAG...,"['▁ATGAAATT', 'TACGATT', 'CAAAAAGATC', 'GTTTA'..."
1,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,NC_014551.1_3,ATGGCAAATCCGATTTCGATTGATACAGAAATGATTACGCTTGGCC...,"['▁ATGGCAAA', 'TCCGATT', 'TCGATTGA', 'TACAGAAA..."
2,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,NC_014551.1_4,TTGTATATCCAAAATTTAGAATTAACATCCTACCGTAATTATGAGC...,"['▁TT', 'GTATATC', 'CAAAATT', 'TA', 'GAATTAA',..."
3,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,NC_014551.1_5,TTGTATATTCATTTAGGTGACGACTTTGTCGTTTCAACACGCGATA...,"['▁TT', 'GTATATT', 'CATTTA', 'GGTGA', 'CGACTT'..."
4,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,NC_014551.1_6,ATGGCTATGGAACAGCAGCAAAATAGTTATGATGAGAATCAGATAC...,"['▁ATGGC', 'TATGGAA', 'CAGCAGC', 'AAAA', 'TAGT..."


In [23]:
len(bpe_data_with_gene_seq)

49806

In [24]:
bpe_data_with_gene_seq.iloc[0]

0    Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...
1                                        NC_014551.1_2
2    ATGAAATTTACGATTCAAAAAGATCGTTTAGTCGAAAGTGTACAAG...
3    ['▁ATGAAATT', 'TACGATT', 'CAAAAAGATC', 'GTTTA'...
Name: 0, dtype: object

### Train word2vec model (on 49000 genes, bpe)

In [2]:
path_w2v_model_bpe = 'dna2vec_data/word2vec_49000genes_bpe_size300.model'

In [14]:
num_features = 300
context_size = 5
min_count = 1
num_workers = multiprocessing.cpu_count()
seed = 42

In [15]:
model_bpe = Word2Vec(size=num_features,
                     window=context_size,
                     min_count=min_count,
                     workers=num_workers,
                     seed=seed)

model_bpe.save(path_w2v_model_bpe)

In [16]:
model_bpe.build_vocab(all_genes_words_bpe)
model_bpe.save(path_w2v_model_bpe)

In [17]:
model_bpe.corpus_count, model_bpe.epochs

(49806, 5)

In [18]:
#about 3 min
start = timer()
model_bpe.train(all_genes_words_bpe, total_examples=model_bpe.corpus_count, epochs=model_bpe.epochs)
model_bpe.save(path_w2v_model_bpe)
print(f'total time: {timer()-start:.2} sec')

total time: 1.8e+02 sec


In [3]:
# just load the model
#model_bpe = Word2Vec.load(path_w2v_model_bpe)

### Read .csv file (1000 genes)

In [27]:
df_genes = pd.read_csv('~/sf_data/all_genes.csv', header=None)
df_genes.shape

(1066, 4)

In [28]:
df_genes = df_genes.iloc[:, :3]
df_genes.head()

,0,1,2
0,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,pgk,ATGAATAAGAAAACAGTAAAAGACATCGACGTAAAAGGCAAAGTCG...
1,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplW,ATGAAAGATCCTCGTGATGTTCTTAAGCGCCCCGTCATTACTGAAC...
2,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplE,ATGAACCGCCTTAAAGAAAAGTACAATAAAGAAATTTCACCTGCTT...
3,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplC,ATGACCAAAGGAATCTTAGGAAGAAAAATTGGTATGACGCAAGTAT...
4,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplF,ATGTCTCGTGTAGGTAAGAAACTGCTTGAGATCCCTTCTGAAGTTA...


In [29]:
df_genes[0].unique()

array(['Bacillus_amyloliquefaciens_GCF_000196735.1_ASM19673v1_genomic',
       'Bacillus_atrophaeus_GCF_000742675.1_ASM74267v1_genomic',
       'Bacillus_halotolerans_GCF_001517105.1_ASM151710v1_genomic',
       'Bacillus_licheniformis_GCF_000011645.1_ASM1164v1_genomic',
       'Bacillus_mojavensis_GCF_000245335.1_ASM24533v1_genomic',
       'Bacillus_paralicheniformis_GCF_000408885.1_ASM40888v1_genomic',
       'Bacillus_siamensis_GCF_000262045.1_KCTC_13613_01_genomic',
       'Bacillus_sonorensis_GCF_002202015.1_ASM220201v1_genomic',
       'Bacillus_subtilis_GCF_000009045.1_ASM904v1_genomic',
       'Bacillus_tequilensis_GCF_000507145.1_KCTC_13622_01_genomic',
       'Bacillus_vallismortis_GCF_000245315.1_ASM24531v1_genomic',
       'Bacillus_velezensis_GCF_002117165.1_ASM211716v1_genomic'],
      dtype=object)

In [30]:
orgs = ['_'.join(org_name.split('_',2)[0:2]) for org_name in df_genes[0]]
len(orgs)

1066

In [31]:
path_gene_nuc = '../../sf_data/prodigal_output/gene_nuc/'
org_names = [f[:f.index('_', f.index('_') + 1)] for f in os.listdir(path_gene_nuc)]
org_names

['Bacillus_amyloliquefaciens',
 'Bacillus_atrophaeus',
 'Bacillus_halotolerans',
 'Bacillus_licheniformis',
 'Bacillus_mojavensis',
 'Bacillus_paralicheniformis',
 'Bacillus_siamensis',
 'Bacillus_sonorensis',
 'Bacillus_subtilis',
 'Bacillus_tequilensis',
 'Bacillus_vallismortis',
 'Bacillus_velezensis']

In [32]:
# check sets of organisms names in two files
print(set(orgs) ^ set(org_names))
print(set(org_names) ^ set(orgs))

set()
set()


In [33]:
tmp_org = []

for name in df_genes[0]:
    for (org_id, org_name) in enumerate(org_names):
        if name.startswith(org_name):
            tmp_org.append('_'.join((str(org_id), org_name)))

In [34]:
df_genes['org_id_name'] = tmp_org
df_genes.head()

,0,1,2,org_id_name
0,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,pgk,ATGAATAAGAAAACAGTAAAAGACATCGACGTAAAAGGCAAAGTCG...,0_Bacillus_amyloliquefaciens
1,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplW,ATGAAAGATCCTCGTGATGTTCTTAAGCGCCCCGTCATTACTGAAC...,0_Bacillus_amyloliquefaciens
2,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplE,ATGAACCGCCTTAAAGAAAAGTACAATAAAGAAATTTCACCTGCTT...,0_Bacillus_amyloliquefaciens
3,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplC,ATGACCAAAGGAATCTTAGGAAGAAAAATTGGTATGACGCAAGTAT...,0_Bacillus_amyloliquefaciens
4,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplF,ATGTCTCGTGTAGGTAAGAAACTGCTTGAGATCCCTTCTGAAGTTA...,0_Bacillus_amyloliquefaciens


In [35]:
del tmp_org

In [36]:
df_genes = df_genes.iloc[:, 1:]
df_genes.head()

,1,2,org_id_name
0,pgk,ATGAATAAGAAAACAGTAAAAGACATCGACGTAAAAGGCAAAGTCG...,0_Bacillus_amyloliquefaciens
1,rplW,ATGAAAGATCCTCGTGATGTTCTTAAGCGCCCCGTCATTACTGAAC...,0_Bacillus_amyloliquefaciens
2,rplE,ATGAACCGCCTTAAAGAAAAGTACAATAAAGAAATTTCACCTGCTT...,0_Bacillus_amyloliquefaciens
3,rplC,ATGACCAAAGGAATCTTAGGAAGAAAAATTGGTATGACGCAAGTAT...,0_Bacillus_amyloliquefaciens
4,rplF,ATGTCTCGTGTAGGTAAGAAACTGCTTGAGATCCCTTCTGAAGTTA...,0_Bacillus_amyloliquefaciens


In [37]:
df_genes.rename(columns={1:'gene_name',2:'gene_seq',3:'org_id_name'}, inplace=True)
df_genes.head()

,gene_name,gene_seq,org_id_name
0,pgk,ATGAATAAGAAAACAGTAAAAGACATCGACGTAAAAGGCAAAGTCG...,0_Bacillus_amyloliquefaciens
1,rplW,ATGAAAGATCCTCGTGATGTTCTTAAGCGCCCCGTCATTACTGAAC...,0_Bacillus_amyloliquefaciens
2,rplE,ATGAACCGCCTTAAAGAAAAGTACAATAAAGAAATTTCACCTGCTT...,0_Bacillus_amyloliquefaciens
3,rplC,ATGACCAAAGGAATCTTAGGAAGAAAAATTGGTATGACGCAAGTAT...,0_Bacillus_amyloliquefaciens
4,rplF,ATGTCTCGTGTAGGTAAGAAACTGCTTGAGATCCCTTCTGAAGTTA...,0_Bacillus_amyloliquefaciens


In [38]:
df_meta_info = df_genes.loc[:, ['org_id_name', 'gene_name']]
df_meta_info.head()

,org_id_name,gene_name
0,0_Bacillus_amyloliquefaciens,pgk
1,0_Bacillus_amyloliquefaciens,rplW
2,0_Bacillus_amyloliquefaciens,rplE
3,0_Bacillus_amyloliquefaciens,rplC
4,0_Bacillus_amyloliquefaciens,rplF


In [39]:
df_genes.head()

,gene_name,gene_seq,org_id_name
0,pgk,ATGAATAAGAAAACAGTAAAAGACATCGACGTAAAAGGCAAAGTCG...,0_Bacillus_amyloliquefaciens
1,rplW,ATGAAAGATCCTCGTGATGTTCTTAAGCGCCCCGTCATTACTGAAC...,0_Bacillus_amyloliquefaciens
2,rplE,ATGAACCGCCTTAAAGAAAAGTACAATAAAGAAATTTCACCTGCTT...,0_Bacillus_amyloliquefaciens
3,rplC,ATGACCAAAGGAATCTTAGGAAGAAAAATTGGTATGACGCAAGTAT...,0_Bacillus_amyloliquefaciens
4,rplF,ATGTCTCGTGTAGGTAAGAAACTGCTTGAGATCCCTTCTGAAGTTA...,0_Bacillus_amyloliquefaciens


### find bpe samples for 1000 genes

Only without the first gene. So there would be not 1066, but 1065 genes

In [40]:
len(df_genes)

1066

In [41]:
bpe_data_with_gene_seq.values.tolist()[0]

['Bacillus_amyloliquefaciens_GCF_000196735.1_ASM19673v1_genomic.fasta',
 'NC_014551.1_2',
 'ATGAAATTTACGATTCAAAAAGATCGTTTAGTCGAAAGTGTACAAGACGTTCTAAAAGCTGTTTCATCCAGAACAACAATTCCCATTCTTACCGGTATTAAGATCGTAGCCTCAGATGAAGGAGTATCATTTACAGGAAGCGACTCTGATATTTCAATCGAGTCATTCATTCCAAAAGAAGAAGATGATAAGGAAATCGTGACGATCGATCAGCCGGGAAGTATCGTGCTGCAAGCCCGTTTCTTCAGTGAAATCGTTAAAAAACTTCCGATGGCGACTGTTGAAATCGAAGTTCAAAATCAATATTTAACGATTATCCGCTTCGGAAAAGCCGAATTTAATTTAAACGGCCTTGATGCAGATGAATATCCTCATCTTCCGCAAATCGAAGAGCATCATGCCATTCAGATTCCGACGGACCTTTTGAAAAACCTAATCCGTCAAACCGTGTTCGCTGTGTCCGCCTCAGAAACACGCCCTATCTTGACAGGTGTAAACTGGAAAGTGGAAAACGGTGAATTATTATGCACTGCCACGGATAGCCATCGACTCGCTTTAAGAAAGGCGAAGCTTGATATTCCGGAAGACAGCTCTTATAACGTCGTCATCCCCGGAAAAAGCTTAACCGAGCTGAGCAAAATTCTTGATGACAGCCAAGAACTGGTAGATATCGTTATTACAGAAACGCAAGTTCTGTTTAAAGCGAAAAACGTATTGTTCTTCTCAAGGCTGCTTGACGGAAATTATCCGGACACGACCAGCTTAATTCCGCAGGAAAGCAAAACAGAAATCGTCGTGAACACGAAGGAATTTTTACAGGCGATAGACCGTGCTTCTTTATTGGCGAGAGAAGGCCGCAACAACGTTGTAAAGCTTTCTGCCAAACCGGCAGACTCGCTTGAGATTTC

In [42]:
i = 1
[(df_genes.iloc[i]['org_id_name'].split('_')[1:], bpe[0].split('_')[:2])
 for bpe in bpe_data_with_gene_seq.values
 if df_genes.iloc[i]['gene_seq'] == bpe[2]
 and df_genes.iloc[i]['org_id_name'].split('_')[1:] == bpe[0].split('_')[:2]]

[(['Bacillus', 'amyloliquefaciens'], ['Bacillus', 'amyloliquefaciens'])]

In [60]:
genes_1000_pre_vec_first_10 = []

start = timer()

for i in range(len(df_genes[:10])):
    [genes_1000_pre_vec_first_10.append((
        bpe[0],
        df_genes.iloc[i]['org_id_name'],
        bpe[1],
        df_genes.iloc[i]['gene_name'],
        df_genes.iloc[i]['gene_seq'],
        bpe[3]))
     for bpe in bpe_data_with_gene_seq.values
     if df_genes.iloc[i]['gene_seq'] == bpe[2]
     and df_genes.iloc[i]['org_id_name'].split('_')[1:] == bpe[0].split('_')[:2]]
    
print(f'total time: {timer()-start:.2} sec')

total time: 1e+02 sec


In [59]:
bpe_data_with_gene_seq.values.shape

(49806, 4)

In [62]:
len(genes_1000_pre_vec_first_10)

10

In [ ]:
# about 3 hours!
genes_1000_pre_vec = []

start = timer()

for i in range(len(df_genes)):
    print(f'{i}: {timer()-start:.2} sec')
    [genes_1000_pre_vec.append((
        bpe[0],
        df_genes.iloc[i]['org_id_name'],
        bpe[1],
        df_genes.iloc[i]['gene_name'],
        df_genes.iloc[i]['gene_seq'],
        bpe[3]))
     for bpe in bpe_data_with_gene_seq.values
     if df_genes.iloc[i]['gene_seq'] == bpe[2]
     and df_genes.iloc[i]['org_id_name'].split('_')[1:] == bpe[0].split('_')[:2]]
    
print(f'total time: {timer()-start:.2} sec')

In [44]:
pd.DataFrame(genes_1000_pre_vec).to_csv('util_data/genes_1000_pre_vec.csv', header=False, index=False)

In [45]:
len(genes_1000_pre_vec)

1065

In [5]:
# load file genes_1000_pre_vec.csv

#genes_1000_pre_vec = pd.read_csv('util_data/genes_1000_pre_vec.csv', header=None, index_col=None)

In [6]:
genes_1000_pre_vec.head()

,0,1,2,3,4,5
0,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,0_Bacillus_amyloliquefaciens,NC_014551.1_3361,pgk,ATGAATAAGAAAACAGTAAAAGACATCGACGTAAAAGGCAAAGTCG...,"['▁ATGAATAA', 'GAAAACA', 'GTAAAA', 'GACATCGA',..."
1,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,0_Bacillus_amyloliquefaciens,NC_014551.1_122,rplW,ATGAAAGATCCTCGTGATGTTCTTAAGCGCCCCGTCATTACTGAAC...,"['▁ATGAAAGA', 'TCCTCGTGA', 'TGTTCTTAA', 'GCGC'..."
2,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,0_Bacillus_amyloliquefaciens,NC_014551.1_132,rplE,ATGAACCGCCTTAAAGAAAAGTACAATAAAGAAATTTCACCTGCTT...,"['▁ATGAACCGC', 'CTTAAAGAAAA', 'GTACAA', 'TAAAG..."
3,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,0_Bacillus_amyloliquefaciens,NC_014551.1_120,rplC,ATGACCAAAGGAATCTTAGGAAGAAAAATTGGTATGACGCAAGTAT...,"['▁ATGA', 'CCAAAGGAA', 'TCTTA', 'GGAA', 'GAAAA..."
4,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,0_Bacillus_amyloliquefaciens,NC_014551.1_135,rplF,ATGTCTCGTGTAGGTAAGAAACTGCTTGAGATCCCTTCTGAAGTTA...,"['▁ATGTC', 'TCGTGTA', 'GGTAA', 'GAAA', 'CTGCTT..."


In [7]:
genes_1000_pre_vec.shape

(1065, 6)

The order is not the same with starting order

### gene2vec (for 1000 genes, bpe)

In [54]:
# wery fast
genes_bpe_vec = []
for gene in genes_1000_pre_vec.values:
    ngrams = gene[5][1:-1].replace(' ','').replace("'",'').split(',')
    genes_bpe_vec.append(np.vstack([model_bpe.wv[ngram] for ngram in ngrams]).mean(axis=0))

In [55]:
len(genes_bpe_vec)

1065

In [56]:
len(genes_bpe_vec[7])

300

In [58]:
pd.DataFrame(genes_bpe_vec).to_csv('dna2vec_data/genes_vec_bpe_1065_genes.tsv', sep='\t', header=False, index=False)

### Create meta_info for bpe

In [38]:
# load file genes_vec_bpe_1065_genes

#genes_bpe_vec = pd.read_csv('dna2vec_data/genes_vec_bpe_1065_genes.tsv', sep='\t', header=None, index_col=None)

In [39]:
genes_bpe_vec.shape

(1065, 300)

In [40]:
genes_bpe_vec.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.039018,0.136491,-0.145523,0.170539,-0.121021,0.016576,-0.023800,-0.129099,-0.245439,-0.051326,...,-0.317607,0.102738,0.083131,-0.023715,0.129599,0.003794,0.192270,-0.030493,-0.071842,-0.081417
1,0.087040,0.012174,-0.091688,0.038298,-0.085748,0.076170,0.047399,-0.081632,-0.297463,-0.072215,...,-0.268071,0.115027,0.099411,0.042407,0.115421,0.004475,0.179053,0.037602,-0.120935,-0.090398
2,0.024723,-0.053478,-0.142268,0.084106,-0.123484,-0.066321,0.056646,-0.120521,-0.287860,-0.051948,...,-0.258727,0.060879,0.045641,0.035888,0.181339,-0.014151,0.217226,0.058193,-0.021986,-0.090381
3,0.062068,0.039552,-0.125375,0.114022,-0.127860,0.008957,0.084016,-0.124281,-0.265910,-0.132501,...,-0.317253,0.092345,0.126914,0.009494,0.125307,-0.018274,0.202105,0.013432,-0.016041,-0.078024
4,0.019426,0.022985,-0.104884,0.074529,-0.142394,0.020855,0.049389,-0.134668,-0.278547,-0.069883,...,-0.299159,0.051952,0.086767,0.058832,0.184349,-0.052133,0.184952,-0.008276,-0.082412,-0.172264


In [5]:
# load file genes_1000_pre_vec.csv

#genes_1000_pre_vec = pd.read_csv('util_data/genes_1000_pre_vec.csv', header=None, index_col=None)

In [6]:
genes_1000_pre_vec.shape

(1065, 6)

In [7]:
genes_1000_pre_vec.head()

,0,1,2,3,4,5
0,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,0_Bacillus_amyloliquefaciens,NC_014551.1_3361,pgk,ATGAATAAGAAAACAGTAAAAGACATCGACGTAAAAGGCAAAGTCG...,"['▁ATGAATAA', 'GAAAACA', 'GTAAAA', 'GACATCGA',..."
1,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,0_Bacillus_amyloliquefaciens,NC_014551.1_122,rplW,ATGAAAGATCCTCGTGATGTTCTTAAGCGCCCCGTCATTACTGAAC...,"['▁ATGAAAGA', 'TCCTCGTGA', 'TGTTCTTAA', 'GCGC'..."
2,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,0_Bacillus_amyloliquefaciens,NC_014551.1_132,rplE,ATGAACCGCCTTAAAGAAAAGTACAATAAAGAAATTTCACCTGCTT...,"['▁ATGAACCGC', 'CTTAAAGAAAA', 'GTACAA', 'TAAAG..."
3,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,0_Bacillus_amyloliquefaciens,NC_014551.1_120,rplC,ATGACCAAAGGAATCTTAGGAAGAAAAATTGGTATGACGCAAGTAT...,"['▁ATGA', 'CCAAAGGAA', 'TCTTA', 'GGAA', 'GAAAA..."
4,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,0_Bacillus_amyloliquefaciens,NC_014551.1_135,rplF,ATGTCTCGTGTAGGTAAGAAACTGCTTGAGATCCCTTCTGAAGTTA...,"['▁ATGTC', 'TCGTGTA', 'GGTAA', 'GAAA', 'CTGCTT..."


In [14]:
meta_bpe = genes_1000_pre_vec.iloc[:, [1,2,3]]

In [ ]:
meta_bpe.rename(columns={1:'org_id_name',2:'gene_id',3:'gene_name'}, inplace=True)

In [20]:
meta_bpe.head()

,org_id_name,gene_id,gene_name
0,0_Bacillus_amyloliquefaciens,NC_014551.1_3361,pgk
1,0_Bacillus_amyloliquefaciens,NC_014551.1_122,rplW
2,0_Bacillus_amyloliquefaciens,NC_014551.1_132,rplE
3,0_Bacillus_amyloliquefaciens,NC_014551.1_120,rplC
4,0_Bacillus_amyloliquefaciens,NC_014551.1_135,rplF


In [22]:
meta_bpe.shape

(1065, 3)

In [23]:
pd.DataFrame(meta_bpe).to_csv('dna2vec_data/meta_bpe_1065_genes.tsv', sep='\t', index=False)

### Clusterization

In [43]:
X_bpe = np.array(genes_bpe_vec)
X_bpe.shape, X_bpe

((1065, 300), array([[ 0.03901764,  0.1364913 , -0.14552276, ..., -0.03049318,
         -0.07184207, -0.08141667],
        [ 0.08703981,  0.01217383, -0.09168836, ...,  0.03760219,
         -0.12093481, -0.09039836],
        [ 0.02472271, -0.05347804, -0.14226761, ...,  0.05819328,
         -0.02198565, -0.09038094],
        ...,
        [ 0.06066027,  0.11064012, -0.20590815, ..., -0.05486792,
         -0.08078935, -0.10742232],
        [-0.00247136, -0.01454612, -0.1557087 , ..., -0.0276449 ,
         -0.0645554 , -0.07112077],
        [ 0.06272714,  0.08192176, -0.1437192 , ...,  0.01054248,
         -0.06647722, -0.06235442]]))

In [28]:
meta_bpe.iloc[:, 2].head()

0     pgk
1    rplW
2    rplE
3    rplC
4    rplF
Name: gene_name, dtype: object

In [60]:
len(meta_bpe.iloc[:, 2].unique())

92

In [30]:
labels_true = np.array(meta_bpe.iloc[:, 2])
len(labels_true), labels_true

(1065,
 array(['pgk', 'rplW', 'rplE', ..., 'tsaD', 'rplD', 'ligA'], dtype=object))

In [44]:
def clst_dbscan(X, labels_true, eps=0.3, min_samples=2):
    print(f'eps={eps}, min_samples={min_samples}')
    clst = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
    core_samples_mask = np.zeros_like(clst.labels_, dtype=bool)
    core_samples_mask[clst.core_sample_indices_] = True

    labels = clst.labels_
    
    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    
    print(f'number of predicted clusters: {n_clusters_}\n')
    print(f'Silhouette_score: {silhouette_score(X, labels)}')
    print(f'Homogeneity: {homogeneity_score(labels_true, labels)}')
    print(f'Completeness: {completeness_score(labels_true, labels)}')
    print(f'V-measure: {v_measure_score(labels_true, labels)}')
    print(f'Adjusted Rand Index: {adjusted_rand_score(labels_true, labels)}')
    print(f'Adjusted Mutual Information: {adjusted_mutual_info_score(labels_true,labels,average_method="arithmetic")}')
    
    return labels

In [66]:
predict_0_6__1 = clst_dbscan(X_bpe, labels_true,
                             eps=0.6, min_samples=1)

eps=0.6, min_samples=1
number of predicted clusters: 98

Silhouette_score: 0.13067525035116098
Homogeneity: 0.37773339138282797
Completeness: 0.8536593095642324
V-measure: 0.5237250892249422
Adjusted Rand Index: 0.01567451799206737
Adjusted Mutual Information: 0.3210951006980934


In [67]:
predict_0_2__2 = clst_dbscan(X_bpe, labels_true,
                             eps=0.2, min_samples=2)

eps=0.2, min_samples=2
number of predicted clusters: 75

Silhouette_score: -0.20078414749606155
Homogeneity: 0.17068969908592
Completeness: 0.6714163467337355
V-measure: 0.2721839007195359
Adjusted Rand Index: 0.001637030518446962
Adjusted Mutual Information: 0.07442426432648629


In [68]:
predict_0_3__2 = clst_dbscan(X_bpe, labels_true,
                             eps=0.3, min_samples=2)

eps=0.3, min_samples=2
number of predicted clusters: 182

Silhouette_score: 0.0033407182010158766
Homogeneity: 0.47805993271283453
Completeness: 0.7014466504195
V-measure: 0.5685996895594548
Adjusted Rand Index: 0.011972899334271024
Adjusted Mutual Information: 0.2551000142283312


In [69]:
predict_0_4__2 = clst_dbscan(X_bpe, labels_true,
                             eps=0.4, min_samples=2)

eps=0.4, min_samples=2
number of predicted clusters: 153

Silhouette_score: 0.09672275923509824
Homogeneity: 0.5400166042062892
Completeness: 0.7538123266214006
V-measure: 0.6292503794462515
Adjusted Rand Index: 0.04240477148180394
Adjusted Mutual Information: 0.3889860309753322


In [70]:
predict_0_5__2 = clst_dbscan(X_bpe, labels_true,
                             eps=0.5, min_samples=2)

eps=0.5, min_samples=2
number of predicted clusters: 114

Silhouette_score: 0.14973479503627646
Homogeneity: 0.5003562868319124
Completeness: 0.8047689593511247
V-measure: 0.6170614037789266
Adjusted Rand Index: 0.031738019196580515
Adjusted Mutual Information: 0.4033202168628336


In [71]:
predict_0_6__2 = clst_dbscan(X_bpe, labels_true,
                             eps=0.6, min_samples=2)

eps=0.6, min_samples=2
number of predicted clusters: 67

Silhouette_score: 0.15025534919288483
Homogeneity: 0.35911593749982007
Completeness: 0.8542722226317756
V-measure: 0.5056630354415088
Adjusted Rand Index: 0.015734014828033313
Adjusted Mutual Information: 0.3193212144322864


In [72]:
predict_0_4__3 = clst_dbscan(X_bpe, labels_true,
                             eps=0.4, min_samples=3)

eps=0.4, min_samples=3
number of predicted clusters: 83

Silhouette_score: 0.02957411535267343
Homogeneity: 0.4155394574378479
Completeness: 0.7742051258081841
V-measure: 0.540809821627749
Adjusted Rand Index: 0.03014539188089236
Adjusted Mutual Information: 0.3514501308451074


In [73]:
predict_0_6__4 = clst_dbscan(X_bpe, labels_true,
                             eps=0.6, min_samples=4)

eps=0.6, min_samples=4
number of predicted clusters: 29

Silhouette_score: 0.12201037456600608
Homogeneity: 0.2925508850899297
Completeness: 0.8776365956098076
V-measure: 0.4388243201498465
Adjusted Rand Index: 0.016448381588399445
Adjusted Mutual Information: 0.3098342552511979


In [74]:
predict_0_5__5 = clst_dbscan(X_bpe, labels_true,
                             eps=0.5, min_samples=5)

eps=0.5, min_samples=5
number of predicted clusters: 27

Silhouette_score: 0.0613025426329341
Homogeneity: 0.29961733679902475
Completeness: 0.8607011477503927
V-measure: 0.4445003490037111
Adjusted Rand Index: 0.027375014043938915
Adjusted Mutual Information: 0.33669600238904135


In [75]:
predict_0_6__5 = clst_dbscan(X_bpe, labels_true,
                             eps=0.6, min_samples=5)

eps=0.6, min_samples=5
number of predicted clusters: 25

Silhouette_score: 0.1081445079219449
Homogeneity: 0.27493997865536707
Completeness: 0.8752836274022989
V-measure: 0.4184411805982626
Adjusted Rand Index: 0.016590607544431018
Adjusted Mutual Information: 0.3007660855069818


In [76]:
meta_bpe.head()

,org_id_name,gene_id,gene_name
0,0_Bacillus_amyloliquefaciens,NC_014551.1_3361,pgk
1,0_Bacillus_amyloliquefaciens,NC_014551.1_122,rplW
2,0_Bacillus_amyloliquefaciens,NC_014551.1_132,rplE
3,0_Bacillus_amyloliquefaciens,NC_014551.1_120,rplC
4,0_Bacillus_amyloliquefaciens,NC_014551.1_135,rplF


In [77]:
meta_bpe_first_preds = meta_bpe.copy()

In [78]:
meta_bpe_first_preds.head()

,org_id_name,gene_id,gene_name
0,0_Bacillus_amyloliquefaciens,NC_014551.1_3361,pgk
1,0_Bacillus_amyloliquefaciens,NC_014551.1_122,rplW
2,0_Bacillus_amyloliquefaciens,NC_014551.1_132,rplE
3,0_Bacillus_amyloliquefaciens,NC_014551.1_120,rplC
4,0_Bacillus_amyloliquefaciens,NC_014551.1_135,rplF


In [79]:
meta_bpe_first_preds['predict_0_6__1'] = predict_0_6__1
meta_bpe_first_preds['predict_0_2__2'] = predict_0_2__2
meta_bpe_first_preds['predict_0_3__2'] = predict_0_3__2
meta_bpe_first_preds['predict_0_4__2'] = predict_0_4__2
meta_bpe_first_preds['predict_0_5__2'] = predict_0_5__2

meta_bpe_first_preds['predict_0_6__2'] = predict_0_6__2
meta_bpe_first_preds['predict_0_4__3'] = predict_0_4__3
meta_bpe_first_preds['predict_0_6__4'] = predict_0_6__4
meta_bpe_first_preds['predict_0_5__5'] = predict_0_5__5
meta_bpe_first_preds['predict_0_6__5'] = predict_0_6__5

In [80]:
meta_bpe_first_preds.head()

,org_id_name,gene_id,gene_name,predict_0_6__1,predict_0_2__2,predict_0_3__2,predict_0_4__2,predict_0_5__2,predict_0_6__2,predict_0_4__3,predict_0_6__4,predict_0_5__5,predict_0_6__5
0,0_Bacillus_amyloliquefaciens,NC_014551.1_3361,pgk,0,-1,-1,0,0,0,0,0,0,0
1,0_Bacillus_amyloliquefaciens,NC_014551.1_122,rplW,1,0,0,1,1,1,1,1,-1,1
2,0_Bacillus_amyloliquefaciens,NC_014551.1_132,rplE,0,-1,-1,2,2,0,2,0,12,0
3,0_Bacillus_amyloliquefaciens,NC_014551.1_120,rplC,0,1,1,3,0,0,3,0,0,0
4,0_Bacillus_amyloliquefaciens,NC_014551.1_135,rplF,0,-1,-1,-1,3,0,-1,0,13,0


In [81]:
pd.DataFrame(meta_bpe_first_preds).to_csv('dna2vec_data/meta_1065genes_bpe_10preds.tsv', sep='\t', index=False)